# EDF Renewables Wind Flow Model Competition Speedup Calculation
*by Florian Roscheck, Data Scientist, Wind Resource Assessment Group, EDF Renewables, 2019-11-06*

This notebook explains how we, the EDF Renewables Wind Resource Assessment Group, calculate mast speedups in the 
Wind Flow Model Competition. It provides the context, the Python function that is used to calculate speedups, as well 
as a ready-to-use-and-play-with example. It is meant to enable you, the contestant, to provide the most appropriate data
for your entry in the Wind Flow Model Competition.  

## Prerequisites

To run this notebook (and the speedup calculation) you should have the following Python libraries and versions 
installed:
```
python==3.7.3
numpy==1.16.4
scipy==1.3.1
faker==2.0.3
pandas==0.24.2
```

## Context: Which Speedups are Calculated?

This section explains which information we provide and which speedups you should calculate.

Every site in the competition has multiple masts. For every site, we calculate the speedups between all mast 
combinations. A mast consists of a speed and a direction time series. We supply measured time series data for only one 
mast per site (the "reference" mast). You are expected to use your wind flow model to predict the wind speed speedups 
relative to that reference mast at all other masts.

We provide mast location, elevation, measurement height, and exposure information (where applicable) for all masts at 
the site. At every mast, there is only one measurement height that you need to predict. That height is usually between
50 and 80 m and specified in the site documents we provide to you. When we score your entry, we only score it against
our measurements at that height.
 
For scoring your model, we weigh errors by direction. This helps you and us understand how well your model captures the
wind flow in the most dominant wind directions. For this reason, we supply a directional time series for the reference
mast and expect you to bin your speedup data by direction. The next section explains in detail how this binning works.

## Calculating Speedups by Total Least Squares Regression
<a id='tls_regression_explanation'></a>

When we score your model, we compare your submitted directional speedups to the ones that we obtained from our measured
data. This section outlines how we obtain those speedups.

We want to calculate the directional speedups between two masts. One of them is the *initiation* mast, 
identified by $I$. The other one is the *target* mast, identified by $T$. Every mast has a speed time series $v$ and a
direction time series $\theta$. That means we have the following time series available: $v_I$, $v_T$, $\theta_I$, 
$\theta_T$. We then perform the following steps:

1. Get concurrent data $v_{I,c}$, $v_{T,c}$, and $\theta_{T,c}$ of $v_I$, $v_T$, and $\theta_T$. We use measured 
    10-min time series.
    
2. Bin $v_{I,c}$ and $v_{T,c}$ by target mast direction $\theta_{T,c}$. There are 24 uniform directional bins $b$ and 
    the first bin is centered at $0^\circ$. So, for example, the first bin $b_0$ would be 
    $352.5^\circ \leq \theta_T < 7.5^\circ$. Binned data is denoted by $v_{I,c,b}$ and $v_{T,c,b}$, respectively.
    
3. For every bin $b$ of $\theta_{T,c}$, by means of total least squares regression, fit a model of the form 
    $y_b = m_b * x_b$ to the speed data, with $x_b$ being $v_{I,c,b}$ and $y_b$ being $v_{T,c,b}$. The slope 
    of the model, $m_b$, is the *directional speedup*. 
  
On our side, we use Scipy's Orthogonal Distance Regression package and the following code for the total least squares
regression:

In [ ]:
# This cell will not return anything, it will just make sure that the fit_tls function is loaded and can be accessed in this notebook
import numpy as np
from scipy.odr import odrpack

def fit_tls(x: np.ndarray, y: np.ndarray) -> float:
    """Fit total least squares model to data

    This function fits a total least squares (also known as orthogonal distance regression) model to 2-dimensional data.
    The model has the form `y = m * x`, where m is the "slope". (It has an intercept at y==0).

    See Also:
        At its core, the function uses scipy's `Orthogonal Distance Regression`_ module

    .. _Orthogonal Distance Regression:
        https://docs.scipy.org/doc/scipy/reference/odr.html

    Args:
        x: 1-dimensional Numpy array
        y: 1-dimensional Numpy array of the same size as `x`

    Returns:
        The slope of the fitted model

    """
    odr_data = odrpack.RealData(x, y)
    model = odrpack.Model(lambda beta, x_: beta[0] * x_)
    odr_container = odrpack.ODR(odr_data, model, beta0=[1.0])
    slope = odr_container.run().beta[0]
    return slope

## Speedup Calculation Example

This section walks you through an actual speedup calculation (albeit with "fake" data). We have validated the code in 
this section with actual measured data, so you can be certain that this is exactly what we are doing behind the scenes 
to come up with our directional speedups that we compare your submitted speedups against.

### Generate Fake Time Series

For this example, we first generate some fake speed and direction time series for initiation and target mast. While no
particular emphasis is placed on making the time series realistic, 

For convenience, we use the [Faker](https://github.com/joke2k/faker) Python package to quickly come up with some time 
series.

In [ ]:
# Initialize and seed Faker
from faker import Faker
fake = Faker('en_US')
fake.seed(42) # Do not change this seed, we rely on it for verifying that you get the same results that we get.

# Initialize and seed numpy
import numpy as np
np.random.seed(42) # Do not change this seed, we rely on it for verifying that you get the same results that we get.

In [ ]:
# Define fake time series generators
from datetime import datetime, timedelta

def generate_spd(weibull_k: float, weibull_c: float, drr: float) -> float:
    if np.random.uniform() < drr:
        return weibull_c*np.random.weibull(weibull_k)
    else:
        return np.nan

def generate_dir(peak: float, spread_std: float, drr: float) -> float:
    if np.random.uniform() < drr:
        value = np.random.normal(peak, spread_std)
        return np.mod(value, 360.0)
    else:
        return np.nan

# Set distributions of values for Faker package to sample from
mast_initiation_spd_distrib = lambda _: generate_spd(weibull_k=2.25, weibull_c=8.25, drr=0.85)
mast_initiation_dir_distrib = lambda _: generate_dir(peak=90.0, spread_std=60.0, drr=0.85)
mast_target_spd_distrib = lambda _: generate_spd(weibull_k=2.0, weibull_c=8.0, drr=0.85)
mast_target_dir_distrib = lambda _: generate_dir(peak=80.0, spread_std=70.0, drr=0.85)

start_date = datetime.strptime('2015-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2019-01-01', '%Y-%m-%d')
time_interval = timedelta(minutes=10)

mast_intiation_spd = fake.time_series(start_date, end_date, time_interval, mast_initiation_spd_distrib)
mast_intiation_dir = fake.time_series(start_date, end_date, time_interval, mast_initiation_dir_distrib)
mast_target_spd = fake.time_series(start_date, end_date, time_interval, mast_target_spd_distrib)
mast_target_dir = fake.time_series(start_date, end_date, time_interval, mast_target_dir_distrib)

In [ ]:
# Generate fake time series
# (Running this cell will take a while)
import pandas as pd

mast_initiation_spd_series = pd.DataFrame(mast_intiation_spd).set_index(0)[1]
print('Generated "mast_initiation_spd_series".')

mast_initiation_dir_series = pd.DataFrame(mast_intiation_dir).set_index(0)[1]
print('Generated "mast_initiation_dir_series".')

mast_target_spd_series = pd.DataFrame(mast_target_spd).set_index(0)[1]
print('Generated "mast_target_spd_series".')

mast_target_dir_series = pd.DataFrame(mast_target_dir).set_index(0)[1]
print('Generated "mast_target_dir_series".')

### Get Concurrent Data

Just as explained [above](#tls_regression_explanation) in step 1, we first need to obtain concurrent data. Note that we
do not need the `mast_initiation_dir_series` for any of the following steps.

In [ ]:
# Put all relevant time series into one Pandas DataFrame
all_masts = pd.concat([mast_initiation_spd_series, mast_target_spd_series, mast_target_dir_series], axis=1)
all_masts.columns = ['SPD_Initiation', 'SPD_Target', 'DIR_Target']
all_masts.head()

In [ ]:
# Filter concurrent data
print('No. of samples before filtering by concurrent timesteps: {}'.format(all_masts.shape[0]))
all_masts_conc = all_masts.dropna().copy()
print('No. of samples after filtering by concurrent timesteps: {}'.format(all_masts_conc.shape[0]))

### Bin Concurrent Data

Now we bin the concurrent speed data by the concurrent target direction.

In [ ]:
no_of_bins = 24
bin_width = 360.0/no_of_bins
all_masts_conc.loc[:, 'Bin'] = np.floor_divide(np.mod(all_masts_conc['DIR_Target'] + (bin_width / 2.0), 360), bin_width) + 1
all_masts_conc.loc[:, 'Bin'] = all_masts_conc.loc[:, 'Bin'].astype(int)
all_masts_conc.head()

### Calculate Speedups

In [ ]:
speedups = {}

for bin_no in np.unique(all_masts_conc['Bin'].values):
    x_in_bin = all_masts_conc[all_masts_conc['Bin'] == bin_no]['SPD_Initiation'].values
    y_in_bin = all_masts_conc[all_masts_conc['Bin'] == bin_no]['SPD_Target'].values
    slope = fit_tls(x=x_in_bin, y=y_in_bin)
    speedups[bin_no] = slope

In [ ]:
speedups = pd.Series(speedups)
speedups

**Great**, these are the speedups!

### Validate Calculation

How do you know that you got exactly the same value that we got? On your path to running this code, some things might 
have taken a wrong turn. For example: You could have accidentally used the wrong NumPy version, or the computer system
that you are working on produces a slightly different result because of its internal configuration.

We want to confirm that when you run the code on your system you get the exact same answer that we got when we run it
on ours. This will give you certainty that you can calculate speedups the exact same way that we can and help you 
optimize your entry in the wind flow model competition.

Run the following cell to know if your results match ours. Note that for the results to match you need to have all cells
executed in order. If you run into trouble, try restarting the kernel and running all cells.

In [ ]:
if abs(0.77664505-speedups.prod()) < 1e-7:
    print('SUCCESS! You can calculate speedups the exact same way that EDF Renewables can.')
else:
    print('ERROR! You are not getting the same results as EDF Renewables. Why don\'t you reach out to Florian to figure'
          ' out what to do next?')